In [1]:
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
dat = pd.read_csv("analysis_result.csv")
dat.head(1)

,entropy,variance,edge_density,b,g,r,edge_diff,edge_entropy,adv
0,6.700422,835.759799,1.319356,43.535091,41.478838,14.986072,25.075529,0.101289,0


In [3]:
len(dat)

7979

In [4]:
dat['adv'].unique()

array([0, 1], dtype=int64)

In [5]:
x = dat.iloc[:, :-1].values
y = dat.iloc[:,-1].values

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)
#y_train = y_train.reshape([len(y_train), 1])
#y_test = y_test.reshape([len(y_test), 1])

In [7]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [9]:
keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None)

clf = Sequential([
    Dense(1024, kernel_initializer='uniform', input_dim=8, activation='elu'),
    Dense(2048, kernel_initializer='uniform', activation='elu'),
    Dense(1024, kernel_initializer='uniform', activation='elu'),
    Dropout(0.1),
    Dense(512, kernel_initializer='uniform', activation='elu'),
    Dense(256, kernel_initializer='uniform', activation='elu'),
    Dense(1, kernel_initializer='uniform', activation='sigmoid')
])

In [10]:
clf.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              9216      
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              2099200   
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_4 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 2

In [11]:
clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
clf.fit(x_train, y_train, batch_size=32, epochs=20)

Epoch 1/20
175/175 [==============================] - 8s 46ms/step - loss: 0.3198 - accuracy: 0.8788

In [13]:
score = clf.evaluate(x_test, y_test, verbose=0)

print(clf.metrics_names)
print(score)

['loss', 'accuracy']
[0.25746792554855347, 0.897243082523346]


In [14]:
from keras.models import load_model
clf.save('AEGuard.h5')

In [45]:
x_predict = [
    [7.3378496842073835,2005.3647338785904,4.504145408163265,32.902195839142415,33.678372772158376,33.41943138869921,75.08849557522123,0.2649477141230245],
    [7.347049941119418,2043.8005363495167,3.0113998724489797,32.896276164881336,33.6823649646088,33.42135887050986,31.436135009927202,0.19496323402345217]
]
x_predict = sc.transform(x_predict)
y_predict = clf.predict(x_predict)
y_predict

array([[0.87222916],
       [0.06468347]], dtype=float32)